In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

In [267]:
def load_data(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    df = yf.Ticker(ticker).history(start=start_date, end=end_date).dropna()
    df.reset_index(inplace=True)
    return df

def add_sma(df: pd.DataFrame, lower: int, upper: int) -> pd.DataFrame:
    if f'sma_{lower}' not in df.columns:
        df[f'sma_{lower}'] = df.Close.rolling(lower).mean()
    if f'sma_{upper}' not in df.columns:
        df[f'sma_{upper}']  = df.Close.rolling(upper).mean()
    return df

def backtest(df: pd.DataFrame, lower: int, upper: int, shares : int, buy_first_day = False) -> pd.DataFrame:
    schema={'date': 'datetime64[ns]', 'action': 'str', 'cash_movement': 'float64'}
    results = pd.DataFrame(columns=schema.keys()).astype(schema)
    if buy_first_day:
        results.loc[len(results)] = [df["Date"].iloc[0], "Buy", df["Close"].iloc[0] * shares * -1]
        waiting_for_bear = True
    else:
        waiting_for_bear = False

    for index, row in df.iterrows():
        if not waiting_for_bear:
            if row[f"sma_{lower}"] > row[f"sma_{upper}"]:
                results.loc[len(results)] = [row.Date, "Buy", row.Close * shares * -1]
                waiting_for_bear = True
        if waiting_for_bear:
            if row[f"sma_{lower}"] < row[f"sma_{upper}"]:
                results.loc[len(results)] = [row.Date, "Sell", row.Close * shares]
                waiting_for_bear = False
    if waiting_for_bear:
        results.loc[len(results)] = [df["Date"].iloc[-1], "Sell", df["Close"].iloc[-1] * shares]
    return results

In [281]:
def run_single_backtest(ticker: str, start_date: str, end_date: str, transaction_costs = 1) -> pd.DataFrame:
    df = load_data(ticker, start_date, end_date)
    shares = round(1000 / df["Close"][0])
    print(f"Shares: {shares}. start price: {df["Close"][0]}. end price: {df["Close"].iloc[-1]}. to beat: {df["Close"].iloc[-1]*shares - df["Close"][0]*shares - transaction_costs}")
    df = add_sma(df, 10, 20)
    res = backtest(df, 10, 20, shares, True)
    gain = res['cash_movement'].sum()
    gain = gain - transaction_costs * len(res['cash_movement'])
    print(f"Gains: {gain})")
    return df


In [282]:
results_nvda = run_single_backtest("NVDA", "2015-01-01", "2025-01-01")

Shares: 2070. start price: 0.48314353823661804. end price: 134.2899932861328. to beat: 276980.1789781451
Gains: 177960.04693460464)


In [6]:
x = pd.DataFrame(np.arange(10, 210, 10))
test_params = pd.merge(x,x,how='cross')
test_params.columns = ['sma_lower','sma_upper']
test_params = test_params[test_params.sma_lower < test_params.sma_upper]
test_params

,sma_lower,sma_upper
1,10,20
2,10,30
3,10,40
4,10,50
5,10,60
...,...,...
338,170,190
339,170,200
358,180,190
359,180,200


In [287]:
def run_multiple_backtests(ticker, investment_sum = 1000, start_date = "2015-01-01", end_date = "2025-01-01"):
    schema={'lower': 'int64', 'upper': 'int64', 'gain': 'float64'}
    results = pd.DataFrame(columns=schema.keys()).astype(schema)
    df = load_data(ticker, start_date, end_date=end_date)
    shares = round(investment_sum / df["Close"][0])
    print(f"Shares: {shares}. start price: {df["Close"][0]}. end price: {df["Close"].iloc[-1]}. to beat: {df["Close"].iloc[-1]*shares - df["Close"][0]*shares}")
    for n,m in test_params.values:
        df = add_sma(df, n, m)
        res = backtest(df, n, m, shares)
        gain = res['cash_movement'].sum()
        transaction_costs = 1
        gain = gain - transaction_costs * len(res['cash_movement'])
        results.loc[len(results)] = [n, m, gain]
    sorted_results = results.sort_values(by='gain', ascending=False)
    return sorted_results

In [288]:
# setup test
ticker = "NVDA"
res_nvda = run_multiple_backtests(ticker)
res_nvda

Shares: 2070. start price: 0.48314353823661804. end price: 134.2899932861328. to beat: 276980.1789781451


,lower,upper,gain
126,90.00,120.00,"309,821.27"
135,100.00,110.00,"301,097.67"
94,60.00,160.00,"300,504.11"
75,50.00,110.00,"298,242.05"
82,50.00,180.00,"297,201.84"
...,...,...,...
7,10.00,90.00,"172,252.39"
4,10.00,60.00,"162,004.52"
5,10.00,70.00,"155,440.76"
6,10.00,80.00,"144,022.33"


In [289]:
ticker = "KO"
res_ko = run_multiple_backtests(ticker)
res_ko

Shares: 33. start price: 30.662113189697266. end price: 62.2599983215332. to beat: 1042.730209350586


,lower,upper,gain
135,100.00,110.00,664.80
123,80.00,200.00,643.77
126,90.00,120.00,626.41
102,70.00,110.00,615.91
111,70.00,200.00,606.47
...,...,...,...
71,50.00,70.00,-124.92
45,30.00,120.00,-159.99
54,40.00,50.00,-184.06
70,50.00,60.00,-198.28


In [290]:
ticker = "MMM"
res_mmm = run_multiple_backtests(ticker)
res_mmm

Shares: 10. start price: 97.4424057006836. end price: 128.4563446044922. to beat: 310.13938903808594


,lower,upper,gain
160,120.00,190.00,814.02
174,140.00,200.00,778.21
167,130.00,190.00,769.53
161,120.00,200.00,764.51
175,150.00,160.00,762.84
...,...,...,...
3,10.00,50.00,-314.33
19,20.00,30.00,-333.87
1,10.00,30.00,-362.10
2,10.00,40.00,-429.01


In [291]:
ticker = "PFE"
res_pfe = run_multiple_backtests(ticker)
res_pfe

Shares: 51. start price: 19.715234756469727. end price: 26.09853744506836. to beat: 325.5484371185303


,lower,upper,gain
151,110.00,180.00,"1,243.55"
159,120.00,180.00,"1,214.98"
152,110.00,190.00,"1,122.02"
177,150.00,180.00,"1,115.04"
132,90.00,180.00,"1,074.28"
...,...,...,...
99,70.00,80.00,"-1,047.79"
58,40.00,90.00,"-1,062.46"
71,50.00,70.00,"-1,091.60"
57,40.00,80.00,"-1,138.06"


In [292]:
res_aapl = run_multiple_backtests("AAPL")
res_aapl

Shares: 41. start price: 24.320430755615234. end price: 250.1449737548828. to beat: 9258.80626296997


,lower,upper,gain
176,150.00,170.00,"10,136.21"
171,140.00,170.00,"9,779.13"
184,170.00,180.00,"9,740.38"
167,130.00,190.00,"9,516.39"
182,160.00,190.00,"9,496.11"
...,...,...,...
85,60.00,70.00,"3,072.51"
128,90.00,140.00,"3,048.65"
125,90.00,110.00,"2,874.68"
145,110.00,120.00,"2,385.71"


In [293]:
res_msft = run_multiple_backtests("MSFT")
res_msft

Shares: 25. start price: 40.0721321105957. end price: 420.6565246582031. to beat: 9514.609813690186


,lower,upper,gain
117,80.00,140.00,"10,240.63"
127,90.00,130.00,"10,224.27"
137,100.00,130.00,"10,221.73"
128,90.00,140.00,"10,208.12"
126,90.00,120.00,"10,141.89"
...,...,...,...
19,20.00,30.00,"3,570.58"
37,30.00,40.00,"3,509.47"
2,10.00,40.00,"3,390.50"
21,20.00,50.00,"3,086.20"


# BUY and Sell

## Interactive

In [ ]:
from ib_insync import *
util.startLoop()
ib =  IB()
ib.connect()

In [ ]:
import pandas as pd

def get_data(market_data):
    print(f"Market Data: {market_data.ask}")
    print(f"askSize: {market_data.askSize}")
    print(f"marketPrice: {market_data.marketPrice()}")
    print(f"marketPrice: {market_data.time}")

contract_apple = Stock("AAPL", "SMART", "USD")
market_data = ib.reqMktData(contract_apple)
get_data(market_data)


In [ ]:
order = MarketOrder(action = "BUY", totalQuantity = 1)
trade = ib.placeOrder(contract_apple, order)

order = LimitOrder(action = "BUY", totalQuantity = 1, lmtPrice = 200)
trade = ib.placeOrder(contract, order)


# Alpaca

In [ ]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest

k = get_secret("broker.alpaca.key")
s = get_secret("broker.alpaca.secret")

trading_client = TradingClient(k, s, paper=False)
account = trading_client.get_account()


In [ ]:
from alpaca.trading.enums import AssetClass, AssetExchange, AssetStatus
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY, exchange=AssetExchange.NASDAQ, status=AssetStatus.ACTIVE)

assets = trading_client.get_all_assets(search_params)
print(f"{len(assets)} found")


In [ ]:
asset = 'SMR'
trading_asset = trading_client.get_asset(asset)

if trading_asset.tradable:
    print(f'We can trade {asset}.')


In [ ]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
symbol = "SMR"
stock_client = StockHistoricalDataClient(k, s)
quote = StockLatestQuoteRequest(symbol_or_symbols=symbol)
latest_multisymbol_quotes = stock_client.get_stock_latest_quote(quote)
latest_ask_price = latest_multisymbol_quotes[symbol].ask_price
print(latest_ask_price)
trading_client = TradingClient(k, s, paper=False)


In [ ]:
from alpaca.trading.requests import OrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderType

order_request = OrderRequest(
                   symbol="SMR",
                   qty = 2,
                   side = OrderSide.BUY,
                   type = OrderType.MARKET,
                   time_in_force = TimeInForce.GTC
                   )

new_order  = trading_client.submit_order(
               order_data=order_request)


In [ ]:
from alpaca.trading.requests import LimitOrderRequest

limit_order_data = LimitOrderRequest(
                    symbol="AMR",
                    limit_price=8,
                    notional=2,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.FOK
                   )
limit_order = trading_client.submit_order(
                order_data=limit_order_data)
